In [160]:
import pandas as pd
import numpy
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
paper = pd.read_csv(r"C:\Users\Admin\Downloads\kdd cup\dataRev\Paper.csv")[:2000]
paper= paper.set_index("Id")
paper['Keyword']= paper['Keyword'].fillna("")
paper['Title']= paper['Title'].fillna("")
title = list(paper["Title"])
paper.head()

,Title,Year,ConferenceId,JournalId,Keyword
Id,,,,,
1,Stitching videos streamed by mobile phones in ...,2009,167,0,mobile video capturing|real-time|video stitching
2,A nonlocal convection–diffusion equation,2007,0,7234,Nonlocal diffusion; Convection–diffusion; Asym...
3,Area Effects in Cepaea,1963,0,16867,
4,Multiple paternity in a natural population of ...,2005,0,6130,
5,Complexity of Finding Short Resolution Proofs,1997,158,0,


In [161]:
word_tokenize(paper.loc[1,'Keyword'])

['mobile', 'video', 'capturing|real-time|video', 'stitching']

In [162]:
#preprocessing keywords
def filter_keyword(text):
    for i in string.punctuation: text = text.replace(i,' ')
    words = word_tokenize(text) #split words
    words = [w.lower() for w in words if w.isalpha()] #get rid of punctuation
    words = [w for w in words if w not in  ["keywords"] ]
    stemmed = [porter.stem(w) for w in words]
    return stemmed
paper['Keyword_pro'] = paper['Keyword'].map(filter_keyword)
    

In [163]:

stopword = set(stopwords.words('english'))
porter = PorterStemmer()
def tokenize(text):
    #text = text.split() # get single words 
    #table = maketrans('','',string.punctuation)
    #stripped = [w.translate(table).lower() for w in text]#get rid of all punctuation
    words = word_tokenize(text) #split words
    words = [w.lower() for w in words if w.isalpha()] #get rid of punctuation
    words =[w for w in words if  not w in stopword]
    stemmed = [porter.stem(w) for w in words]
    return stemmed
paper['Token'] = paper.Title.map(tokenize)


In [164]:
#concatenate keyword and token
paper['Key_token'] = paper[['Keyword_pro','Token']].apply((lambda x: ' '.join(list(set([i for z in x for i in z])))), axis =1)

In [167]:
token = list(paper['Key_token'])
count = CountVectorizer(min_df = 5)
tfidf = TfidfTransformer()
count_token =count.fit_transform(token).toarray() #2000*527
#tfid_token = tfidf.fit_transform(count_token)
vocab = list(count.vocabulary_.keys())



In [168]:
#list of common words in each title of each document
paper['Common word'] = paper['Key_token'].map(lambda x: [i for i in x.split() if i in vocab])
paper.loc[:5,'Common word']
    

Id
1    [real, stream, time, mobil, video]
2                       [diffus, equat]
3                        [effect, area]
4       [storag, natur, multipl, popul]
5              [find, resolut, complex]
Name: Common word, dtype: object